In [5]:
!pip install docling docling-core python-dotenv langchain-text-splitters langchain-ollama langchain-milvus

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip available: 22.3 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [1]:
from typing import Iterator
from langchain_core.document_loaders import BaseLoader
from langchain_core.documents import Document as LCDocument
from docling.document_converter import DocumentConverter

class DoclingPDFLoader(BaseLoader):
    def __init__(self, file_path: str | list[str]) -> None:
        self._file_paths = file_path if isinstance(file_path, list) else [file_path]
        self._converter = DocumentConverter()

    def lazy_load(self) -> Iterator[LCDocument]:
        for source in self._file_paths:
            dl_doc = self._converter.convert(source).document
            text = dl_doc.export_to_markdown()
            yield LCDocument(page_content=text)

In [11]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader = DoclingPDFLoader(file_path="../data/docling-tech-report.pdf")
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
)

docs = loader.load()
splits = text_splitter.split_documents(docs)

In [12]:
from langchain_huggingface.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(
    model_name="BAAI/bge-small-en-v1.5"
)

In [13]:
from langchain_milvus import Milvus
URI = "./milvus_example.db"
vectorstore = Milvus.from_documents(
    splits,
    embeddings,
    connection_args={"uri": URI},
    drop_old=True,
)

In [14]:
from langchain_ollama import OllamaLLM

llm = OllamaLLM(
    model="vanilj/Phi-4:latest"
)

In [15]:
from typing import Iterable
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough

def format_docs(docs: Iterable[LCDocument]):
    return "\n\n".join(doc.page_content for doc in docs)

retriever = vectorstore.as_retriever()

prompt = PromptTemplate.from_template("""
Context information is below.
---------------------
{context}
---------------------
Given the context information and not prior knowledge, answer the query.
Query: {question}
Answer:
""")

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [16]:
response = rag_chain.invoke("Which AI models have been released by docling team?")
print(response)

The Docling team has released two highly capable AI models to the open-source community:

1. **Layout Analysis Model**: This model is an accurate object-detector for page elements.

2. **TableFormer**: A state-of-the-art table structure recognition model.

Both of these models are part of the Docling package and are designed to facilitate document conversion tasks efficiently on commodity hardware. The pre-trained weights and inference code packages for these models are hosted on Hugging Face as `docling-ibm-models`.
